In [ ]:
# %%python -m pip install --upgrade pip
# %pip install opencv-python
# %pip install mediapipe
# %pip install wget
# %pip install pillow
# %pip install pyaudio


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\timcd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\timcd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\timcd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\timcd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\timcd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# !python -m wget https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task


Saved under hand_landmarker (50).task


In [10]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [ ]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

def flip_lr_names(name):
  match name:
    case "Left":
      return "Right"
    case "Right":
      return "Left"

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for handedness, hand_landmarks in zip(handedness_list, hand_landmarks_list):
    # hand_landmarks = hand_landmarks_list[idx]
    # handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.

    cv2.putText(annotated_image, f"{flip_lr_names(handedness[0].category_name)}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [12]:
import numpy as np
import pyaudio


class SineOscillator:
    def __init__(self, sample_rate, duration, frequency_factor=1, volume_factor=1):
        self.fs = sample_rate  
        self.duration = duration
        self.frequency_factor = frequency_factor
        self.volume_factor = volume_factor

    def __call__(self, frequency):
        return (np.sin(2 * np.pi * np.arange(self.fs * self.duration) * frequency * self.frequency_factor / self.fs)).astype(np.float32) * self.volume_factor
    
def hanning_window(length, volume):
    window = np.hanning(length).astype(np.float32)

    return window * volume

def rect_window(length, volume):
    fade_duration = 0.002
    fade_samples = int(self.fs * fade_duration)
    base = np.ones(length).astype(np.float32) * volume
    base[:fade_samples] = np.linspace(0, volume, fade_samples)
    base[-fade_samples:] = np.linspace(volume, 0, fade_samples)
    return base

class Synthesizer:
    def __init__(self, sampling_rate, duration):
        self.fs = sampling_rate
        self.duration = duration
        self.previous_frequency = None
        self.previous_volume = None
    
    def get_volume(self, volume):
        if volume is None and self.previous_volume is not None:
            volume = self.previous_volume
        else :
            self.previous_volume = volume
        return volume
    
    def get_frequency(self, frequency):
        if frequency is None and self.previous_frequency is not None:
            frequency = self.previous_frequency
        else :
            self.previous_frequency = frequency
        return frequency

class SineSynthesizer(Synthesizer):
    def __init__(self, sampling_rate, duration):
        super().__init__(sampling_rate)
        self.oscillator = SineOscillator(self.fs, self.duration)
        self.window = hanning_window  
        
    def get_waveform(self, frequency, volume):
        volume = self.get_volume(volume)
        frequency = self.get_frequency(frequency)

        if frequency is None or volume is None:
            return # nothing I can do
        
        samples = self.oscillator(frequency)
        window = self.window(len(samples), volume)
        return samples * window

class DoubleSineSynthesizer(Synthesizer):
    def __init__(self, sampling_rate, duration):
        super().__init__(sampling_rate, duration)
        self.oscillator = SineOscillator(self.fs, self.duration)
        self.oscillator2 = SineOscillator(self.fs, self.duration, 0.5, 0.5)
        self.window = hanning_window  
    
    def get_waveform(self, frequency, volume):
        volume = self.get_volume(volume)
        frequency = self.get_frequency(frequency)

        if frequency is None or volume is None:
            return # nothing I can do
        
        samples1 = self.oscillator(frequency)
        samples2 = self.oscillator2(frequency)
        window = self.window(len(samples1), volume)
        return (samples1 + samples2) * window/2

class Player:
    def __init__(self):
        self.p = pyaudio.PyAudio()
        self.fs = 44100  # sampling rate, Hz, must be integer
        self.synth = DoubleSineSynthesizer(self.fs, 0.19)
        self.duration = 0.19
        self.stream = self.p.open(format=pyaudio.paFloat32,
                                channels=1,
                                rate=self.fs,
                                output=True)


    def play_note(self, frequency, volume):
        waveform = self.synth.get_waveform(frequency, volume)
        if waveform is None:
            return
        output_bytes = (waveform).tobytes() 
        self.stream.write(output_bytes)

    def __del__(self):
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()

In [13]:
from statistics import mean


def get_hands(detection_result):
    # DEFAULT_VOLUME = 1.0
    KEY = 261.63
    SCALE_FACTOR = 72
    HALF_TONE = 2 ** (1/12)

    left_points = None
    right_points = None

    for handedness, hand_landmarks in zip(detection_result.handedness, detection_result.hand_landmarks):
        if handedness[0].category_name == 'Left':
            right_points = hand_landmarks
        else:
            left_points = hand_landmarks

    volume = mean(point.y for point in left_points) if left_points else None
    x_mean = mean(point.x for point in right_points) if right_points else None
    if x_mean is None:
        return None, volume
    freq = KEY * HALF_TONE ** (SCALE_FACTOR * (x_mean - 0.25))
    if volume is None:
        return freq, None
    return freq, volume
    

In [16]:
webcam = 1
window_name = f"test {webcam}"

# 20 ms = 0.02s 
# 1/0.02 = 50


synth = Player()

# Create Landmarker
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

cv2.namedWindow(window_name)
vc = cv2.VideoCapture(webcam)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    rval, frame = vc.read()

    frame = cv2.flip(frame, 1)
    image = mp.Image(data=np.array(frame), image_format=mp.ImageFormat.SRGB)

    detection_result = detector.detect(image)

    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    
    cv2.imshow(window_name, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2RGBA))
    
    key = cv2.waitKey(1)
    if key in [27, 32]: # exit on ESC or SPACE
        break

    freq, volume = get_hands(detection_result) 
    synth.play_note(freq, volume)
        
    
vc.release()
cv2.destroyWindow(window_name)
del synth